In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
from scipy import signal, fft
from subprocess import run
import re
import pandas as pd
import obspy
import obspy.signal.freqattributes as freqatt
from obspy.core.util.attribdict import AttribDict

In [2]:
### set up variables for this run ###
# run name should correspond to the mesh name given to inparam_mesh/the directory name passed to movemesh.csh
run_name = "CadekEPSCBW2_deep"

# directory containing the AxiSEM SOLVER folder
sim_dir = "~/Documents/Synthetics"

# processed seismic traces will be stored in f"{working_dir}/{run_name}/"
working_dir = "~/Documents/Enceladus"

# name of receiver file used in the simulation
station = "recfile"

# which component to plot
component = "Z"

# extract delta from the recorded simulation data
f = open(f'{sim_dir}/SOLVER/{run_name}/Data_Postprocessing/info_matlab.dat')
lines = f.readlines()
line = lines[2].replace(' ', '')
delta = line.replace('\n','')

# create directory to store processed seismic data
run(['mkdir', '-p', f"{working_dir}/{run_name}"])

In [4]:
#function to read a 2-column file and return x and y which are lists of floats
def Read_2Col(file_name):
    print(file_name)
    with open(file_name, 'r') as data:
        x = []
        y = []
        for line in data:
            p = line.split()
            x.append(float(p[0]))
            y.append(float(p[1]))
            
    return x, y

In [5]:
#function to return station information from the seismogram file chosen including network, latitude + longitude, elevation, and depth
def Read_Station(file_name, station):
        with open(file_name, 'r') as data:
            net = []
            lat = []
            lon = []
            elev = []
            depth = []
            for line in data:
                #find the line with the station name in
                if station in line:
                    p = line.split()
                    print(p)
                    net = p[1]
                    lat = p[2]
                    lon = p[3]
                    elev = p[4]
                    depth = p[5]
                    
        return net, lat, lon, elev, depth

In [6]:
#function to read station information from receivers.dat file type
def Read_Receivers(file_name):
    lat = {}
    lon = {}
    with open(file_name, 'r') as data:
        data = data.readlines()
        n = data[0]
        for i in range(1, len(data)):
            #linesplit
            data[i] = data[i].replace('/n', '')
            line = data[i].split()
            lat[str(i).zfill(4)] = str(90 - float(line[0])) # i+1 used to avoid plotting the first and last stations at 0 and 180 degrees
            lon[str(i).zfill(4)] = line[1]
            
    return lat, lon, n

In [7]:
#function to read source data from source file, reutrns depth, lat+lon as strings
def Read_Source(file_name):
        with open(file_name, 'r') as data:
            lat = []
            lon = []
            dep = []
            for line in data:
                if 'SOURCE_DEPTH' in line:
                    p = line.split()
                    dep = p[1]
                if 'SOURCE_LAT' in line:
                    p = line.split()
                    lat = p[1]
                if 'SOURCE_LON' in line:
                    p = line.split()
                    lon = p[1]
                    
        return dep, lat, lon

In [8]:
#function to find dominant period and background model for this run - file name should be location of inparam_mesh, outputs dominant period as a float
def Read_mesh(file_name):
    with open(file_name, 'r') as data :
        DPeriod = []
        bg_model = []
        ext_model = []
        for line in data :
            if 'DOMINANT_PERIOD' in line :
                p = line.split()
                DPeriod = float(p[1])
            if 'BACKGROUND_MODEL' in line :
                p = line.split()
                bg_model = p[1]
            #external model has a # at the start of the line if it's not in use; ext_model therefore = 'EXT_MODEL' when not in use
            if  'EXT_MODEL' in line :
                p = line.split()
                ext_model = p[1]
            
    return DPeriod, bg_model, ext_model

In [9]:
# read station data
stalat, stalon, n = Read_Receivers(f"{sim_dir}/SOLVER/{run_name}/receivers.dat")
print(stalat)
print(stalon)

# read in the seismogram and assign to time + displacement
t = {}
disp = {}
for station in stalat.keys():
    file_path = f"{sim_dir}/SOLVER/{run_name}/Data_Postprocessing/SEISMOGRAMS/recfile_{station}_disp_post_mij_conv0000_{component}.dat"
    t[station], disp[station] = Read_2Col(file_path)

# read dominant period and background model
DPeriod, bg_model, ext_model = Read_mesh(f"{sim_dir}/SOLVER/{run_name}/inparam_mesh")
print(DPeriod)

{'0001': '90.0', '0002': '82.5', '0003': '75.0', '0004': '67.5', '0005': '60.0', '0006': '52.5', '0007': '45.0', '0008': '37.5', '0009': '30.0', '0010': '22.5', '0011': '15.0', '0012': '7.5', '0013': '0.0', '0014': '-7.5', '0015': '-15.0', '0016': '-22.5', '0017': '-30.0', '0018': '-37.5', '0019': '-45.0', '0020': '-52.5', '0021': '-60.0', '0022': '-67.5', '0023': '-75.0', '0024': '-82.5', '0025': '-90.0'}
{'0001': '0.0', '0002': '0.0', '0003': '0.0', '0004': '0.0', '0005': '0.0', '0006': '0.0', '0007': '0.0', '0008': '0.0', '0009': '0.0', '0010': '0.0', '0011': '0.0', '0012': '0.0', '0013': '0.0', '0014': '0.0', '0015': '0.0', '0016': '0.0', '0017': '0.0', '0018': '0.0', '0019': '0.0', '0020': '0.0', '0021': '0.0', '0022': '0.0', '0023': '0.0', '0024': '0.0', '0025': '0.0'}
/Users/katdapre/Documents/Synthetics/parallel-axisem/SOLVER/CadekEPSCBrantutW2/Data_Postprocessing/SEISMOGRAMS/recfile_0001_disp_post_mij_conv0000_Z.dat
/Users/katdapre/Documents/Synthetics/parallel-axisem/SOLVER/C

In [10]:
# read source data
source_dep, evtlat, evtlon = Read_Source(f"{sim_dir}/SOLVER/{run_name}/inparam_source")

source_dep = str(source_dep)
evtlat = str(evtlat)
evtlon = str(evtlon)

90.0
/Users/katdapre/Documents/Enceladus/Cadek16Enc/CadekEPSCBrantutW2.nd
5.0


In [ ]:
### construct filter ###

sps = 1/float(delta) # samples per second
order = 4             # Order of the filter (corners)
filter_type = 'bandpass'  # Filter type (e.g., 'highpass', 'lowpass', 'bandpass')
band_cutoff = [1,6]       # Critical frequency or frequencies for filter in Hz (use [min_freq, max_freq] for 'bandpass')

# Construct Butterworth filter
sos = signal.butter(order, band_cutoff, btype=filter_type, fs=sps, output='sos')


In [15]:
### process + write data ###

#for each station in the displacement dict:
for s2plot in list(disp.keys()):
    df = pd.DataFrame(plotdisp[s2plot])
    
    # write processed unfiltered data
    amp2 = obspy.Trace(data = np.array(df[0]).T)
    amp2.detrend('simple')
    amp2.detrend('demean')
    amp2.taper(0.05,type='hann')
    amp2.stats['sampling_rate'] = sps
    amp2.stats['station'] = s2plot
    amp2.write(f"{working_dir}/{run_name}/{s2plot}{component}raw.sac")
        
    # write processed filtered data
    f_data = signal.sosfiltfilt(sos, amp2, padtype='odd')
    f_data2 = obspy.Trace(data = np.array(f_data))
    f_data2.stats['delta'] = delta
    f_data2.stats['station'] = s2plot
    f_data2.write(f"{working_dir}/{run_name}/{s2plot}{component}filt_{band_cutoff[0]}_{band_cutoff[1]}.sac")

3.758260440270297e-05
8.478939535527486e-05
8.663648148980066e-06
1.5112267051796343e-06
3.1690285800661975e-07
2.588595562754421e-07
2.186952775080507e-07
1.8746665509461873e-07
1.630128712644188e-07
1.5925203570024663e-07
1.426872566481103e-07
2.7020014811235784e-07
3.595512467461762e-07
4.2011744892170333e-07
4.4745969605574935e-07
4.5663577975902324e-07
4.5674205322599324e-07
4.515009452766693e-07
4.441504735763384e-07
4.361145453979483e-07
4.28609932479142e-07
4.225767264598708e-07
4.1790783983989543e-07
4.153271763991335e-07
3.3489436864195482e-06
